In [ ]:
!pip3 install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 28.0MB/s 
     |████████████████████████████████| 102kB 12.4MB/s 
     |████████████████████████████████| 7.6MB 45.4MB/s 
     |████████████████████████████████| 81kB 12.8MB/s 
  Created wheel for boto3: filename=boto3-1.17.78-py2.py3-none-any.whl size=128921 sha256=e63ff9f5bda8cfe758102eedcb2439ef0ee65af41c98978d79b1a9ccfec185ec
  Stored in directory: /root/.cache/pip/wheels/35/4e/0b/65ce88240e97b158d4a062630f6815337a79cd64bbc61bbd1e
Successfully built boto3
ERROR: botocore 1.20.78 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Russian Bert installation 
!wget -q --show-progress -c "http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_pt.tar.gz"
!tar -zxf rubert_cased_L-12_H-768_A-12_pt.tar.gz

BERT_MODEL_PATH = "rubert_cased_L-12_H-768_A-12_pt"
!cp {BERT_MODEL_PATH}/bert_config.json {BERT_MODEL_PATH}/config.json

rubert_cased_L-12_H 100%[===================>] 631.36M  5.68MB/s    in 1m 55s  


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 40.3MB/s 
     |████████████████████████████████| 3.3MB 45.2MB/s 
     |████████████████████████████████| 901kB 45.8MB/s 


In [ ]:
# Load pre-trained model (weights)
#model_version = 'bert-base-uncased'
model = BertForMaskedLM.from_pretrained(BERT_MODEL_PATH).to(DEVICE)
model.eval()
cuda = torch.cuda.is_available()
if cuda:
    model = model.cuda()

# Load pre-trained model tokenizer (vocabulary)
#tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=model_version.endswith("uncased"))

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH)

def tokenize_batch(batch):
    return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]

def untokenize_batch(batch):
    return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]

def detokenize(sent):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    new_sent = []
    for i, tok in enumerate(sent):
        if tok.startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

CLS = '[CLS]'
SEP = '[SEP]'
MASK = '[MASK]'
mask_id = tokenizer.convert_tokens_to_ids([MASK])[0]
sep_id = tokenizer.convert_tokens_to_ids([SEP])[0]
cls_id = tokenizer.convert_tokens_to_ids([CLS])[0]

In [ ]:
def generate_step(out, gen_idx, temperature=None, top_k=0, sample=False, return_list=True):
    """ Generate a word from from out[gen_idx]
    
    args:
        - out (torch.Tensor): tensor of logits of size batch_size x seq_len x vocab_size
        - gen_idx (int): location for which to generate for
        - top_k (int): if >0, only sample from the top k most probable words
        - sample (Bool): if True, sample from full distribution. Overridden by top_k 
    """
    logits = out[:, gen_idx]
    if temperature is not None:
        logits = logits / temperature
    if top_k > 0:
        kth_vals, kth_idx = logits.topk(top_k, dim=-1)
        dist = torch.distributions.categorical.Categorical(logits=kth_vals)
        idx = kth_idx.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1)
    elif sample:
        dist = torch.distributions.categorical.Categorical(logits=logits)
        idx = dist.sample().squeeze(-1)
    else:
        idx = torch.argmax(logits, dim=-1)
    return idx.tolist() if return_list else idx
  
  
def get_init_text(seed_text, max_len, batch_size = 1, rand_init=False):
    """ Get initial sentence by padding seed_text with either masks or random words to max_len """
    batch = [seed_text + [MASK] * max_len + [SEP] for _ in range(batch_size)]
    #if rand_init:
    #    for ii in range(max_len):
    #        init_idx[seed_len+ii] = np.random.randint(0, len(tokenizer.vocab))
    
    return tokenize_batch(batch)

def printer(sent, should_detokenize=True):
    if should_detokenize:
        sent = detokenize(sent)[1:-1]
    print(" ".join(sent))


In [ ]:
import math
import time

def parallel_sequential_generation(seed_text, batch_size=10, max_len=15, top_k=0, temperature=None, max_iter=300, burnin=200,
                                   cuda=False, print_every=10, verbose=True):
    """ Generate for one random position at a timestep
    
    args:
        - burnin: during burn-in period, sample from full distribution; afterwards take argmax
    """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_iter):
        kk = np.random.randint(0, max_len)
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = mask_id
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        topk = top_k if (ii >= burnin) else 0
        idxs = generate_step(out, gen_idx=seed_len+kk, top_k=topk, temperature=temperature, sample=(ii < burnin))
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = idxs[jj]
            
        if verbose and np.mod(ii+1, print_every) == 0:
            for_print = tokenizer.convert_ids_to_tokens(batch[0])
            for_print = for_print[:seed_len+kk+1] + ['(*)'] + for_print[seed_len+kk+1:]
            print("iter", ii+1, " ".join(for_print))
            
    return untokenize_batch(batch)

def parallel_generation(seed_text, batch_size=10, max_len=15, top_k=0, temperature=None, max_iter=300, sample=True, 
                        cuda=False, print_every=10, verbose=True):
    """ Generate for all positions at each time step """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_iter):
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        for kk in range(max_len):
            idxs = generate_step(out, gen_idx=seed_len+kk, top_k=top_k, temperature=temperature, sample=sample)
            for jj in range(batch_size):
                batch[jj][seed_len+kk] = idxs[jj]
            
        if verbose and np.mod(ii, print_every) == 0:
            print("iter", ii+1, " ".join(tokenizer.convert_ids_to_tokens(batch[0])))
    
    return untokenize_batch(batch)
            
def sequential_generation(seed_text, batch_size=10, max_len=15, leed_out_len=15, 
                          top_k=0, temperature=None, sample=True, cuda=False):
    """ Generate one word at a time, in L->R order """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, batch_size)
    
    for ii in range(max_len):
        inp = [sent[:seed_len+ii+leed_out_len]+[sep_id] for sent in batch]
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        idxs = generate_step(out, gen_idx=seed_len+ii, top_k=top_k, temperature=temperature, sample=sample)
        for jj in range(batch_size):
            batch[jj][seed_len+ii] = idxs[jj]
        
    return untokenize_batch(batch)


def generate(n_samples, seed_text="[CLS]", batch_size=10, max_len=25, 
             generation_mode="parallel-sequential",
             sample=True, top_k=100, temperature=1.0, burnin=200, max_iter=500,
             cuda=False, print_every=1):
    sentences = []
    n_batches = math.ceil(n_samples / batch_size)
    start_time = time.time()
    for batch_n in range(n_batches):
        if generation_mode == "parallel-sequential":
            batch = parallel_sequential_generation(seed_text, batch_size=batch_size, max_len=max_len, top_k=top_k,
                                                   temperature=temperature, burnin=burnin, max_iter=max_iter, 
                                                   cuda=cuda, verbose=False)
        elif generation_mode == "sequential":
            batch = sequential_generation(seed_text, batch_size=batch_size, max_len=max_len, top_k=top_k, 
                                          temperature=temperature, leed_out_len=leed_out_len, sample=sample,
                                          cuda=cuda)
        elif generation_mode == "parallel":
            batch = parallel_generation(seed_text, batch_size=batch_size,
                                        max_len=max_len, top_k=top_k, temperature=temperature, 
                                        sample=sample, max_iter=max_iter, 
                                        cuda=cuda, verbose=False)
        
        if (batch_n + 1) % print_every == 0:
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time))
            start_time = time.time()
        
        sentences += batch
    return sentences

Let's call the actual generation function! We'll use the following settings
- max_len (40): length of sequence to generate
- top_k (100): at each step, sample from the top_k most likely words
- temperature (1.0): smoothing parameter for the next word distribution. Higher means more like uniform; lower means more peaky
- burnin (250): for non-sequential generation, for the first burnin steps, sample from the entire next word distribution, instead of top_k
- max_iter (500): number of iterations to run for
- seed_text (["CLS"]): prefix to generate for. We found it crucial to start with the CLS token; you can try adding to it 

In [ ]:
n_samples = 5
batch_size = 5
max_len = 40
top_k = 100
temperature = 1.0
generation_mode = "parallel-sequential"
leed_out_len = 5
burnin = 250
sample = True
max_iter = 500

seed_text = "[CLS] Он не выносит мусор , хотя он не выносит и".split()
bert_sents = generate(n_samples, seed_text=seed_text, batch_size=batch_size, 
                      max_len=max_len, generation_mode=generation_mode,
                      sample=sample, top_k=top_k, temperature=temperature, 
                      burnin=burnin, max_iter=max_iter, cuda=cuda)

Finished batch 1 in 20.271s


In [ ]:
for sent in bert_sents:
  printer(sent, should_detokenize=True)

Он не выносит мусор , хотя он не выносит и мусора очень много , но также она его политика , что есть люди и есть то , что им происходит Хотя она как раз , так была , и была очень высокого уровня на 1980 ' s Pavement .
Он не выносит мусор , хотя он не выносит и того и другого , и , соответственно , не выносит любых , важных для данного « осмысленного » человека ценностей , ценностей ( и проч . ) , поскольку они включают в себя религиозные ценности и религиозные ценности .
Он не выносит мусор , хотя он не выносит и отходы . ГОДЖИЩЕ : Раньше были и компактные , и просторные кондиционеры по сравнению с которыми раньше . Есть и универсалы , даже и у дробжей , и у моделей .
Он не выносит мусор , хотя он не выносит и 10 - сантиметрового дубравы . Показательно , что во время сна у него есть некоторые приёмы одежды , так как он носит Восемь Порогов по Коллекциону и Семь Блокнотов по журналу .
Он не выносит мусор , хотя он не выносит и работу , и воду , ' " ' - известный тренер пензенцев . ' " О

In [ ]:
seed_text = "[CLS] Он не принимает такую позицию , хотя он не принимает и".split()
bert_sents = generate(n_samples, seed_text=seed_text, batch_size=batch_size, max_len=max_len,
                      generation_mode=generation_mode,
                      sample=sample, top_k=top_k, temperature=temperature, burnin=burnin, max_iter=max_iter,
                      cuda=cuda)

Finished batch 1 in 20.731s


In [ ]:
for sent in bert_sents:
  printer(sent, should_detokenize=True)

Он не принимает такую позицию , хотя он не принимает и на территорию , где теряются преступления национальных меньшинств , неконтролируемое население , территорию , где отменяется действие Устава ООН относительно Сирии , а также в регионе каналов TACIA , контрольно - измерительная инфраструктура .
Он не принимает такую позицию , хотя он не принимает и её . « Ликвида » — название организации , поставленной с целью вывести себя в риск , риск , и пр . Для примера « Ликвид » действует в частном жилом доме , Ликвид — в притоне .
Он не принимает такую позицию , хотя он не принимает и другую . Они люди другой личности . . . ) Религия « Акумусский меч » , 1927 . Наука , которая должна была следовать принципам и положениям наук и технологий , ныне называется эмпириографией .
Он не принимает такую позицию , хотя он не принимает и с противоположной и с противоположной стороны , что исключает развитие языковых норм при приёме на военную службу лиц без воинских званий . " Терминология : / Сравнител